# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6a8804b190>
├── 'a' --> tensor([[ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275]])
└── 'x' --> <FastTreeValue 0x7f6a8804b9d0>
    └── 'c' --> tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                        [ 0.9306,  0.6534, -0.1116, -0.6769],
                        [ 0.3375, -0.8603, -0.4360,  0.5511]])

In [4]:
t.a

tensor([[ 0.1629, -0.9539, -1.1075],
        [ 0.6176, -0.6526, -2.1275]])

In [5]:
%timeit t.a

68.9 ns ± 0.0311 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6a8804b190>
├── 'a' --> tensor([[ 0.9998,  1.1814, -0.4530],
│                   [ 0.1246, -1.0769, -1.2605]])
└── 'x' --> <FastTreeValue 0x7f6a8804b9d0>
    └── 'c' --> tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                        [ 0.9306,  0.6534, -0.1116, -0.6769],
                        [ 0.3375, -0.8603, -0.4360,  0.5511]])

In [7]:
%timeit t.a = new_value

70.3 ns ± 0.234 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275]]),
    x: Batch(
           c: tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                      [ 0.9306,  0.6534, -0.1116, -0.6769],
                      [ 0.3375, -0.8603, -0.4360,  0.5511]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1629, -0.9539, -1.1075],
        [ 0.6176, -0.6526, -2.1275]])

In [11]:
%timeit b.a

58.6 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5743, -1.9309,  0.7959],
               [ 0.7681,  0.9567,  0.3011]]),
    x: Batch(
           c: tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                      [ 0.9306,  0.6534, -0.1116, -0.6769],
                      [ 0.3375, -0.8603, -0.4360,  0.5511]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.329 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.949 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 23.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 532 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f69ddcc9d00>
├── 'a' --> tensor([[[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]],
│           
│                   [[ 0.1629, -0.9539, -1.1075],
│                    [ 0.6176, -0.6526, -2.1275]]])
└── 'x' --> <FastTreeValue 0x7f69ddcc9eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 63.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f69ddcaaa00>
├── 'a' --> tensor([[ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275],
│                   [ 0.1629, -0.9539, -1.1075],
│                   [ 0.6176, -0.6526, -2.1275]])
└── 'x' --> <FastTreeValue 0x7f69ddcaa460>
    └── 'c' --> tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                        [ 0.9306,  0.6534, -0.1116, -0.6769],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 54.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55 µs ± 44.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]],
       
               [[ 0.1629, -0.9539, -1.1075],
                [ 0.6176, -0.6526, -2.1275]]]),
    x: Batch(
           c: tensor([[[-1.0223, -0.5904,  2.3272, -0.4114],
                       [ 0.9306,  0.6534, -0.1116, -0.6769],
                       [ 0.3375, -0.8603, -0.4360,  0.5511]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275],
               [ 0.1629, -0.9539, -1.1075],
               [ 0.6176, -0.6526, -2.1275]]),
    x: Batch(
           c: tensor([[-1.0223, -0.5904,  2.3272, -0.4114],
                      [ 0.9306,  0.6534, -0.1116, -0.6769],
                      [ 0.3375, -0.8603, -0.4360,  0.5511],
                      [-1.0223, -0.5904,  2.3272, -0.4114],
                      [ 0.9306,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

339 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
